In [ ]:
# !wget https://fmi-unibuc-ia.github.io/ia/Data/data_lab6.zip

In [ ]:
# !unzip data_lab6.zip

In [ ]:
import numpy as np
from sklearn.utils import shuffle
# load training data
training_data = np.load('data-lab6/training_data.npy')
prices = np.load('data-lab6/prices.npy')
# print the first 4 samples
print('The first 4 samples are:\n ', training_data[:4])
print('The first 4 prices are:\n ', prices[:4])
# shuffle
training_data, prices = shuffle(training_data, prices, random_state=0)

In [ ]:
import sklearn.preprocessing as preprocessing

def normalize(train_data, test_data, type=None):
    if type is None:
        return train_data, test_data
    elif type == 'standard':
        scaler = preprocessing.StandardScaler()
        scaler.fit(train_data)
        return scaler.transform(train_data), scaler.transform(test_data)
    elif type == 'min-max':
        scaler = preprocessing.MinMaxScaler()
        scaler.fit(train_data)
        return scaler.transform(train_data), scaler.transform(test_data)
    elif type == 'l1' or type == 'l2':
        scaler = preprocessing.Normalizer(type)
        scaler.fit(train_data)
        return scaler.transform(train_data), scaler.transform(test_data)

In [ ]:
import matplotlib.pyplot as plt

for i in range(training_data.shape[1]):
    plt.scatter(training_data[:,i], prices)
    plt.show()

In [6]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso


def train_and_validate(train_data, train_labels, n_folds, modelclass, alpha=None):
    kf = KFold(n_splits=n_folds)

    kf.get_n_splits(train_data)

    mses = []
    maes = []

    for train_index, test_index in kf.split(train_data):
        X_train, X_test = train_data[train_index], train_data[test_index]
        y_train, y_test = train_labels[train_index], train_labels[test_index]

        X_train_scaled, X_test_scaled = normalize(X_train, X_test, 'standard')

        if alpha is None:
            model = modelclass()
        else:
            model = modelclass(alpha=alpha)

        model.fit(X_train_scaled, y_train)

        predictions = model.predict(X_test_scaled)

        # print(model.coef_)

        # calculam MSE, MAE
        mse_value = mean_squared_error(y_test, predictions)
        mae_value = mean_absolute_error(y_test, predictions)

        mses.append(mse_value)
        maes.append(mae_value)

    # print(mses)
    # print(maes)

    return np.mean(maes), np.mean(mses)

print(train_and_validate(training_data, prices, 3, LinearRegression))

print(train_and_validate(training_data, prices, 3, Lasso, 1.1))

print(train_and_validate(training_data, prices, 3, Ridge, 1.1))

(1.3203744, 3.1690786)
(2.0192263, 6.7669826)
(1.3195815, 3.1674192)


In [13]:
mse_min = []
mae_min = []
for alpha in [0.001, 0.01, 0.1, 1]:
  mse, mae = train_and_validate(training_data, prices, 3, Ridge, alpha)

  
#   print(mse, mae)
  mse_min.append((mse, alpha))
  mae_min.append((mae,alpha))

print(min(mse_min))
print(min(mae_min))

(1.3195841, 1)
(3.1674213, 1)


In [ ]:
alpha = 10
scaler = preprocessing.StandardScaler()
scaler.fit(training_data)
X_train_scaled = scaler.transform(training_data)


